# All the fine tuning codes are from https://github.com/brevdev/notebooks/blob/main/llama2-finetune.ipynb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Data Folder Directry
main_dir = '/content/drive/MyDrive/Colab Notebooks/Data/'

In [ ]:
import os
import pandas as pd

In [ ]:
sub_dir = 'Ubuntu_Dialogue_Corpus/Ubuntu-dialogue-corpus/dialogueText_301.csv'
full_file_path = os.path.join(main_dir, sub_dir)

df_ubuntu = pd.read_csv(full_file_path)
df_ubuntu

,folder,dialogueID,date,from,to,text
0,301,1.tsv,2004-11-23T11:49:00.000Z,stuNNed,NaN,any ideas why java plugin takes so long to load?
1,301,1.tsv,2004-11-23T11:49:00.000Z,crimsun,stuNNed,java 1.4?
2,301,1.tsv,2004-11-23T11:49:00.000Z,stuNNed,crimsun,yes
3,301,1.tsv,2004-11-23T11:49:00.000Z,crimsun,stuNNed,java 1.5 loads _much_ faster
4,301,1.tsv,2004-11-23T11:50:00.000Z,stuNNed,crimsun,noneus: how can i get 1.5 is there a .deb some...
...,...,...,...,...,...,...
16587825,32,1783.tsv,2007-11-15T03:38:00.000Z,koyo001,NaN,thanks
16587826,32,1783.tsv,2007-11-15T03:39:00.000Z,koyo001,NaN,does anyone know something
16587827,32,1783.tsv,2007-11-15T03:39:00.000Z,neverblue,NaN,"no, no one knows everything"
16587828,32,1783.tsv,2007-11-15T03:40:00.000Z,koyo001,ikonia,the camera doesnt work


In [ ]:
grouped = df_ubuntu.groupby('dialogueID')
dialogue_pairs = []

for _, group in grouped:
    messages = group['text'].tolist()
    for i in range(len(messages) - 1):
        dialogue_pairs.append({
            'input': str(messages[i]),
            'output': str(messages[i + 1])
        })

In [ ]:
train_size = int(0.8 * len(dialogue_pairs))
train_pairs = dialogue_pairs[:train_size]
eval_pairs = dialogue_pairs[train_size:]

In [ ]:
import json
save_path = '/content/drive/MyDrive/Colab Notebooks/Data/'
# Write training pairs to `ubuntu_train.jsonl` in the specified directory
with open(save_path + 'ubuntu_train.jsonl', 'w') as file:
    for pair in train_pairs:
        file.write(json.dumps(pair) + '\n')

# Write evaluation pairs to `ubuntu_eval.jsonl` in the specified directory
with open(save_path + 'ubuntu_eval.jsonl', 'w') as file:
    for pair in eval_pairs:
        file.write(json.dumps(pair) + '\n')


In [ ]:
import json
save_path = '/content/drive/MyDrive/Colab Notebooks/Data/'

In [ ]:
!pip install -q huggingface_hub
!pip install -q -U trl transformers accelerate peft
!pip install -q -U datasets bitsandbytes einops wandb
!pip install  -q ipywidgets
!pip install -q scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 98.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
from datasets import load_dataset

train_dataset = load_dataset('json', data_files=save_path + 'ubuntu_train.jsonl', split='train')
eval_dataset = load_dataset('json', data_files=save_path + 'ubuntu_eval.jsonl', split='train')



Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
def formatting_func(example):
    text = f"### Context: {example['input']}\n ### Reply: {example['output']}"
    return [text]

In [ ]:
base_model_name = "meta-llama/Llama-2-7b-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)
base_model.config.use_cache = False

# More info: https://github.com/huggingface/transformers/pull/24906
base_model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
output_dir = "./Llama-2-7b-hf-fine-tune-baby"

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=50,
    max_steps=1000,
    logging_dir="./logs",        # Directory for storing logs
    save_strategy="steps",       # Save the model checkpoint every logging step
    save_steps=50,                # Save checkpoints every 50 steps
    evaluation_strategy="steps", # Evaluate the model every logging step
    eval_steps=50,               # Evaluate and save checkpoints every 50 steps
    do_eval=True                 # Perform evaluation at the end of training
)

In [ ]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
max_seq_length = 512
trainer = SFTTrainer(
    model=base_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    formatting_func=formatting_func,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_args,
)


# pass in resume_from_checkpoint=True to resume from a checkpoint
trainer.train()

In [ ]:
import os
output_dir1 = "/content/drive/MyDrive/Colab Notebooks/Data/"
output_dir1 = os.path.join(output_dir1, "final_checkpoint")
trainer.model.save_pretrained(output_dir1)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
from peft import PeftModel
import os
output_dir1 = "/content/drive/MyDrive/Colab Notebooks/Data/"
output_dir1 = os.path.join(output_dir1, "final_checkpoint")

In [ ]:
base_model_name="meta-llama/Llama-2-7b-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
model = PeftModel.from_pretrained(base_model, output_dir1)


In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer


In [ ]:
eval_dataset = load_dataset('json', data_files='/content/drive/MyDrive/Colab Notebooks/Data/ubuntu_eval.jsonl', split='train')


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
model.to("cuda")
model.eval()

responses = []

for example in eval_dataset:
    # Create the prompt based on the 'input' from the dataset
    eval_prompt = f"### Context: {example['input']}\n ### Reply:"

    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        generated_response = tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True)
        responses.append(generated_response)


In [ ]:
!pip install -q rouge_score

In [ ]:
from datasets import load_metric
model.to("cuda")
model.eval()

responses = []

# Convert the first 100 examples into a list of dictionaries
subset_eval = eval_dataset.select(range(1)).with_format("python")

for example in subset_eval:
    # Check if example is a dictionary
    if not isinstance(example, dict):
        print(f"Unexpected data type within dataset: {type(example)}")
        continue  # Skip this iteration if it's not a dictionary

    # Ensure 'input' and 'output' keys exist
    if 'input' not in example or 'output' not in example:
        print(f"Missing keys in example: {example.keys()}")
        continue  # Skip this iteration if keys are missing

    eval_prompt = f"### Context: {example['input']}\n ### Reply:"

    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        generated_response = tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True)
        responses.append(generated_response)

# Extract the actual responses (ground truths) for the first 100 examples
ground_truths = [example['output'] for example in subset_eval if 'output' in example]

# Compute BLEU and ROUGE metrics
bleu_metric = load_metric('bleu')
#rouge_metric = load_metric('rouge')

bleu_score = bleu_metric.compute(predictions=responses, references=[[gt] for gt in ground_truths])
#rouge_score = rouge_metric.compute(predictions=responses, references=ground_truths)

print("BLEU Score:", bleu_score)
#print("ROUGE Score:", rouge_score)





In [ ]:
print(responses)

["### Context: yes\n ### Reply:  that's the whole point of the program, it's a live CD that you can boot up from a CD and install to your hard drive.  I have done that myself, but it's a lot of work.  I'm not sure if there is a way to install the program to a USB stick, but I don't think there is.  The program is designed to be used on a CD, and then you install the rest of the system from the"]


In [ ]:
print(example)

{'input': 'yes', 'output': 'looking now...'}


In [ ]:
from datasets import load_metric

In [ ]:
ground_truths = [example['output'] for example in eval_dataset]


In [ ]:
# Load the metrics
bleu_metric = load_metric('bleu')
rouge_metric = load_metric('rouge')

# Compute BLEU
bleu_score = bleu_metric.compute(predictions=responses, references=[[gt] for gt in ground_truths])

# Compute ROUGE
rouge_score = rouge_metric.compute(predictions=responses, references=ground_truths)

print("BLEU Score:", bleu_score)
print("ROUGE Score:", rouge_score)


In [ ]:
eval_prompt = "What is the command to install a package in Ubuntu?"

# Tokenize the prompt
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

# Set the model to evaluation mode
model.eval()

# Generate a response with a strict token limit
with torch.no_grad():
    generated_response = model.generate(**model_input, max_new_tokens=100)[0]
    decoded_response = tokenizer.decode(generated_response, skip_special_tokens=True)

# Print the provided question and the model's generated response
print("Question:", eval_prompt)
print("Answer:", decoded_response.replace(eval_prompt, "").strip())





Question: What is the command to install a package in Ubuntu?
Answer: I'm trying to install a package in Ubuntu and I don't know what command to use.  

Answer: `sudo apt-get install <package_name>`

Comment: I'm trying to install a package called 'wine' but it says 'wine is already the newest version'

Comment: then run `sudo apt-get install wine-stable`

Comment:
